<a href="https://colab.research.google.com/github/Yanina-Kutovaya/RecSys-retail/blob/main/notebooks/service_test_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Yanina-Kutovaya/RecSys-retail.git
!pip install -r RecSys-retail/requirements_Colab.txt

Cloning into 'RecSys-retail'...
remote: Enumerating objects: 1666, done.
remote: Counting objects: 100% (534/534), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 1666 (delta 266), reused 483 (delta 226), pack-reused 1132
Receiving objects: 100% (1666/1666), 364.01 KiB | 17.33 MiB/s, done.
Resolving deltas: 100% (868/868), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 74 kB/s 
     |████████████████████████████████| 72 kB 490 kB/s 
     |████████████████████████████████| 2.0 MB 51.9 MB/s 
     |████████████████████████████████| 76.6 MB 26 kB/s 
     |████████████████████████████████| 55 kB 1.2 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 80 kB 10.6 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled light

In [2]:
%cd RecSys-retail

/content/RecSys-retail


In [3]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "service"))

import logging
import pandas as pd
from fastapi.testclient import TestClient

from main import app, Model
from src.recsys_retail.models.serialize import load
from src.recsys_retail.data.make_dataset import load_data
from src.recsys_retail.models.train import data_preprocessing_pipeline
from scripts.train_save_model import train_store

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
logger = logging.getLogger(__name__)

client = TestClient(app)

MODEL = os.getenv("MODEL", default="baseline_v1")

In [6]:
def test_healthcheck():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json()["status"] == "Green"
    logger.info(f'status_code = 200, status = "Green"')

In [7]:
def test_predict():
    data, item_features, user_features = load_data()
    train_dataset_lvl_2 = data_preprocessing_pipeline(
        data, item_features, user_features
    )
    train_store(train_dataset_lvl_2, "baseline_v1")
    Model.classifier = load(MODEL)

    user = {"user_id": 1340}
    response = client.post("/predict?user_id=1340", json=user)
    assert response.status_code == 200
    assert response.json()[0]["user_id"] == 1340

    logger.info(
        f"test_predict single user status_code = 200, response: {response.json()}"
    )

    users = {"user_ids": [1340, 1364]}
    response = client.post("/predict_user_list?batch_id=1", json=users)
    assert response.status_code == 200
    assert response.json()[0]["user_ids"] == [1340, 1364]

    logger.info(
        f"test_predict user_list status_code = 200, response: {response.json()}"
    )

In [8]:
test_healthcheck()
test_predict()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2501 [00:00<?, ?it/s]

0:	test: 0.6711423	best: 0.6711423 (0)	total: 131ms	remaining: 2m 10s
50:	test: 0.7902189	best: 0.7902189 (49)	total: 3.99s	remaining: 1m 14s
100:	test: 0.7923090	best: 0.7928367 (96)	total: 7.81s	remaining: 1m 9s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.7928366796
bestIteration = 96

Shrink model to first 97 iterations.
